In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import secrets
import ae352_quadrotor_new

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

In [ ]:
import sympy as sym
import numpy as np
import scipy.signal as signal
from control.matlab import ctrb
from scipy import linalg
import time

pi=np.pi

np.set_printoptions(suppress=True)


In [ ]:
params = {
    'm': 0.5, #kg
    'Jx': 0.0023,
    'Jy': 0.0023,
    'Jz': 0.0040,
    'l': 0.175,
    'g': 9.81,
}

In [ ]:
# components of position (meters)
p_x, p_y, p_z = sym.symbols('p_x, p_y, p_z')

# yaw, pitch, roll angles (radians)
psi, theta, phi = sym.symbols('psi, theta, phi')

# components of linear velocity (meters / second)
v_x, v_y, v_z = sym.symbols('v_x, v_y, v_z')
v_in_body = sym.Matrix([v_x, v_y, v_z])

# components of angular velocity (radians / second)
w_x, w_y, w_z = sym.symbols('w_x, w_y, w_z')
w_in_body = sym.Matrix([w_x, w_y, w_z])

# components of net rotor torque
tau_x, tau_y, tau_z = sym.symbols('tau_x, tau_y, tau_z')

# net rotor force
f_z = sym.symbols('f_z')

# parameters
m = sym.nsimplify(params['m'])
Jx = sym.nsimplify(params['Jx'])
Jy = sym.nsimplify(params['Jy'])
Jz = sym.nsimplify(params['Jz'])
l = sym.nsimplify(params['l'])
g = sym.nsimplify(params['g'])
J = sym.diag(Jx, Jy, Jz)

# rotation matrices
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0], [sym.sin(psi), sym.cos(psi), 0], [0, 0, 1]])
Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)], [0, 1, 0], [-sym.sin(theta), 0, sym.cos(theta)]])
Rx = sym.Matrix([[1, 0, 0], [0, sym.cos(phi), -sym.sin(phi)], [0, sym.sin(phi), sym.cos(phi)]])
R_body_in_world = Rz @ Ry @ Rx

# angular velocity to angular rates
ex = sym.Matrix([[1], [0], [0]])
ey = sym.Matrix([[0], [1], [0]])
ez = sym.Matrix([[0], [0], [1]])
M = sym.simplify(sym.Matrix.hstack((Ry @ Rx).T @ ez, Rx.T @ ey, ex).inv(), full=True)

# applied forces
f_in_body = R_body_in_world.T @ sym.Matrix([[0], [0], [-m * g]]) + sym.Matrix([[0], [0], [f_z]])

# applied torques
tau_in_body = sym.Matrix([[tau_x], [tau_y], [tau_z]])

# equations of motion
f = sym.Matrix.vstack(
    R_body_in_world @ v_in_body,
    M @ w_in_body,
    (1 / m) * (f_in_body - w_in_body.cross(m * v_in_body)),
    J.inv() @ (tau_in_body - w_in_body.cross(J @ w_in_body)),
)

f = sym.simplify(f, full=True)
f

In [ ]:
#DEFINE EQUILIBRIUM
#f at equilibrium

#position\
(p_x_e, p_y_e, p_z_e) = (0,0,0)

#yaw, pitch, roll
(psi_e, theta_e, phi_e) = (0,0,0)

#linear velocity
(v_x_e, v_y_e, v_z_e) = (0,0,0)

#angular velocity
(w_x_e, w_y_e, w_z_e) = (0,0,0)

#net rotor torque
(tau_x_e, tau_y_e, tau_z_e) = (0,0,0)

#net rotor force
(f_z_e) = (m*9.81) #Net rotor force has to counteract gravity for equilibrium to be hover

#f at equilibrium
f_e=f.subs({p_x:p_x_e, 
            p_y:p_y_e,
            p_z:p_z_e, #substituting values in f with equilibrium values
            psi:psi_e, 
            theta:theta_e,
            phi:phi_e,
            v_x:v_x_e, 
            v_y:v_y_e, 
            v_z:v_z_e,
            w_x:w_x_e, 
            w_y:w_y_e, 
            w_z:w_z_e,
            tau_x:tau_x_e, 
            tau_y:tau_y_e, 
            tau_z:tau_z_e,
            f_z:f_z_e})

f_e #f_e is all 0, so this equilibrium state works

In [ ]:
# Position of markers in body frame Acknowledgement to GRAYKS3248 Condynsate Template

p_in_world = sym.Matrix([p_x, p_y, p_z])

lxy=0.25 #x or y distance from quadrotor frame to markers in m CHANGE TO OUR MODEL DISTANCE
lz=0.046875 # The z distance from the quadrotor frame to the markers in m CHANGE TO OUR DISTANCE


mark1_inB = sym.Matrix([[lxy],
                        [0.],
                        [lz]])
mark2_inB = sym.Matrix([[0.],
                        [lxy],
                        [lz]])
mark3_inB = sym.Matrix([[-lxy],
                        [0.],
                        [lz]])
mark4_inB = sym.Matrix([[0.],
                        [-lxy],
                        [lz]])

# Position of markers in world frame
mark1_inW = p_in_world + R_body_in_world @ mark1_inB
mark2_inW = p_in_world + R_body_in_world @ mark2_inB
mark3_inW = p_in_world + R_body_in_world @ mark3_inB
mark4_inW = p_in_world + R_body_in_world @ mark4_inB

# Sensor model
g = sym.simplify(sym.Matrix.vstack(mark1_inW, mark2_inW, mark3_inW, mark4_inW))

g

In [ ]:
#DEFINE EQUILIBRIUM

#g at equilibrium, equilibrium values defined from earlier

g_e=g.subs({p_x:p_x_e, 
            p_y:p_y_e,
            p_z:p_z_e,
            psi:psi_e, 
            theta:theta_e,
            phi:phi_e,
            v_x:v_x_e, 
            v_y:v_y_e, 
            v_z:v_z_e,
            w_x:w_x_e, 
            w_y:w_y_e, 
            w_z:w_z_e,})

g_e=np.array(g_e)
g_e

In [ ]:
#FIND A AT EQUILIBRIUM

#lambdify to create numerical function, 
#then take jacobian of that wrt states
A_num=sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), f.jacobian([p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z])) 

A=A_num(p_x_e,p_y_e,p_z_e,psi_e,theta_e,phi_e,v_x_e,v_y_e,v_z_e,w_x_e,w_y_e,w_z_e,tau_x_e,tau_y_e,tau_z_e,f_z_e) #plugging in equilibrium values into the new numerical equation
sym.Matrix(A)

In [ ]:
#FIND B AT EQULIBRIUM

B_num =  sym.lambdify((p_x, p_y, p_z,psi, theta, phi,v_x, v_y, v_z,w_x, w_y, w_z,tau_x, tau_y, tau_z, f_z), f.jacobian([tau_x, tau_y, tau_z, f_z]))

B = B_num(p_x_e, p_y_e, p_z_e, psi_e, theta_e, phi_e, v_x_e, v_y_e, v_z_e, w_x_e, w_y_e, w_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
sym.Matrix(B)

In [ ]:
#FIND C

C_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), g.jacobian([p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z]))
#Jacobian wrt states, "Cx"

C = C_num(p_x_e,p_y_e,p_z_e,psi_e,theta_e,phi_e,v_x_e,v_y_e,v_z_e,w_x_e,w_y_e,w_z_e,tau_x_e,tau_y_e,tau_z_e,f_z_e)

sym.Matrix(C)

In [ ]:
#FIND D 

D_num = sym.lambdify((p_x,p_y,p_z,psi,theta,phi,v_x,v_y,v_z,w_x,w_y,w_z,tau_x,tau_y,tau_z,f_z), g.jacobian([tau_x,tau_y,tau_z,f_z]))
#Jacobian wrt inputs, "Du"

D = D_num(p_x_e,p_y_e,p_z_e,psi_e,theta_e,phi_e,v_x_e,v_y_e,v_z_e,w_x_e,w_y_e,w_z_e,tau_x_e,tau_y_e,tau_z_e,f_z_e)

sym.Matrix(D)

In [ ]:
#TEST CONTROLLABILITY

W = ctrb(A, B) #Controllability matrix
print(W)

rank_W = np.linalg.matrix_rank(W)
print(rank_W)
print(W.shape) #If the rank of the matrix equals its number of rows (or columns, whichever is smaller),
               #it is full rank and controllable
#It is controllable

In [ ]:
import control as ctrl

O = ctrl.obsv(A, C)  # Observability matrix

print(O)

Orank = np.linalg.matrix_rank(O)
print(Orank)
print(O.shape) #If the rank of the matrix equals its number of columns,
               #it is full rank and observable
#It is observable

In [ ]:
#LQR TO FIND K (FOR CONTROLLER)

def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A,B,Q,R)
    K = np.linalg.inv(R) @ B.T @ P
    return K

#Choose weights
# Q = np.diag([5, 5, 5, 90, 90, 90, 1, 1, 1, 1, 1, 1])
Q = np.diag([1, 1, 1, 90, 90, 90, 1, 1, 1, 1, 1, 1]) #Q VALUES FOR PERFORMANCE GOAL 3
R = np.diag([1000, 1000, 1000, 1])

K = lqr(A,B,Q,R)
sym.Matrix(K)

In [ ]:
#CHECK A-BK REAL PARTS NEGATIVE

xdot=A-B@K

control_eig=np.linalg.eigvals(xdot)

control_eig #the optimized eigenvalues
#these are all real part neg so the system is stable

In [ ]:
#FIND L (FOR OBSERVER)

Q_L = np.identity(C.shape[0])
R_L = np.identity(A.shape[0])

Qinv = np.linalg.inv(Q_L)
Rinv = np.linalg.inv(R_L)

L = lqr(A.T, C.T, Rinv, Qinv).T
# L.shape
sym.Matrix(L)

In [ ]:
#CHECK A-LC REAL PARTS NEGATIVE

xdot_err=A-L@C
# print(xdot_err)

observe_eig=np.linalg.eigvals(xdot_err)
observe_eig #the optimized eigenvalues

#these are all real part neg so the system is detectable and asymptotically stable

xdot_err

In [ ]:
#UNCOMMENT EACH SECTION ONE AT A TIME FOR EACH PERFORMANCE GOAL. RECOMMENT THIS ONE BEFORE RUNNING NEXT
#PERFORMANCE GOAL 1
#Hover 1m above the ground for 2 minutes

class Controller:
    def __init__(self):

        self.dt = 0.01

        self.variables_to_log = ['x_hat', 'x_desired']

        self.A = np.array(A, dtype=float)
        self.B = np.array(B, dtype=float)
        self.C = np.array(C, dtype=float)
        self.K = np.array(K, dtype=float)
        self.L = np.array(L, dtype=float)



        self.tau_x_e = 0
        self.tau_y_e = 0
        self.tau_z_e = 0
        self.f_z_e = 9.81*0.5

    def reset(self, p_x=0, p_y=0, p_z=0, yaw=0):
        self.x_hat = np.array([p_x, p_y, p_z, yaw, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
        self.x_desired = np.zeros(12, dtype=float)

    def run(self, **kwargs):

        mocap1=kwargs['mocap_1']
        mocap2=kwargs['mocap_2']
        mocap3=kwargs['mocap_3']
        mocap4=kwargs['mocap_4']
        
        dt = kwargs.get('dt', 0.01)

        pos_markers = np.array([mocap1[0],mocap1[1],mocap1[2],
                                mocap2[0],mocap2[1],mocap2[2],
                                mocap3[0],mocap3[1],mocap3[2],
                                mocap4[0],mocap4[1],mocap4[2],])
        
        x_desired = np.zeros(12)
        pos = self.x_hat[0:3]
        final_pos = np.array([0, 0, 1])

# -----------------------------------------
        #Hover at 1m above the ground
        e_max = 0.01
    
        if np.linalg.norm(final_pos - pos) > 0.1:
            p_desired = pos + e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
            v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
        elif np.linalg.norm(final_pos - pos) > 0.05:
            p_desired = pos + e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
            v_desired = 0.05 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
        elif np.linalg.norm(final_pos - pos) > 0.01:
            p_desired = pos + e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
            v_desired = 0.007 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
        else:
            p_desired = final_pos
            v_desired = 0.001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
# -----------------------------------------
        
        x_desired[0:3] = p_desired
        x_desired[6:9] = v_desired
        self.x_desired = x_desired
    
        u = -self.K @ (self.x_hat - self.x_desired)
    
        y = pos_markers
    
        self.x_hat += dt * (self.A @ self.x_hat + self.B @ u - self.L @ (self.C @ self.x_hat - y))
    
        tau_x = u[0]
        tau_y = u[1]
        tau_z = u[2]
        f_z = u[3] + self.f_z_e
    
        return tau_x, tau_y, tau_z, f_z

In [ ]:
# #PERFORMANCE GOAL 2
# #Fly in a circle of radius 2m at an altitude of 1m above ground at speed of 0.5 m/s for at least 1 min

# class Controller:
#     def __init__(self):

#         self.dt = 0.01

#         self.variables_to_log = ['x_hat', 'x_desired']

#         self.A = A
#         self.B = B
#         self.C = C
#         self.K = K
#         self.L = L

#         self.tau_x_e = 0
#         self.tau_y_e = 0
#         self.tau_z_e = 0
#         self.f_z_e = 9.81*0.5
       
#         self.phase=0 #phase 0 - go to 1m altitude, phase 1 - go forward 2m, phase 2 - rotate around origin
#         self.theta=3*np.pi/2 #starting angle when rotation begins
  
#     def get_color(self):

#         return [
#             1. , # <-- how much red (between 0 and 1)
#             0., # <-- how much green (between 0 and 1)
#             0.25, # <-- how much blue (between 0 and 1)
#         ]

#     def reset(self, p_x=0, p_y=0, p_z=0, yaw=0):
#         self.x_hat = np.array([p_x, p_y, p_z, yaw, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
#         self.x_desired = np.zeros(12, dtype=float)

#     def run(self, **kwargs):

#         mocap1=kwargs['mocap_1']
#         mocap2=kwargs['mocap_2']
#         mocap3=kwargs['mocap_3']
#         mocap4=kwargs['mocap_4']
        
#         dt = kwargs.get('dt', 0.01)

#         pos_markers = np.array([mocap1[0],mocap1[1],mocap1[2],
#                                 mocap2[0],mocap2[1],mocap2[2],
#                                 mocap3[0],mocap3[1],mocap3[2],
#                                 mocap4[0],mocap4[1],mocap4[2],])
        
#         x_desired = np.zeros(12)
#         pos = self.x_hat[0:3]

# # -----------------------------------------
#         #Getting the drone to 1m 

#         e_max = 0.01

#         if self.phase==0:
#             final_pos=np.array([0, 0, 1])
#             if np.linalg.norm(final_pos - pos) > 0.1:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 0.05:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.05 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.01:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.007 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))  

#             else:
#                 p_desired = final_pos
#                 v_desired = np.zeros(3)
#                 self.phase = 1
# # -----------------------------------------
#         #Have frone fly in circle of 2m at 0.5 m/s
        
#         elif self.phase==1:
#             radius=2
#             center=np.array([0,2,1])
#             speed=0.5

#             omega=speed/radius #angular velocity w=v/R

#             self.theta += omega*self.dt

#             self.theta = self.theta % (2*np.pi)

#             x=center[0]+radius*np.cos(self.theta)
#             y=center[1]+radius*np.sin(self.theta)

#             final_pos=np.array([x,y,1]) #altitude is constant

#             if np.linalg.norm(final_pos - pos) > 0.01:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos)) 
                
#             else:
#                 p_desired = final_pos
#                 v_desired = np.zeros(3)
# # -----------------------------------------
        
#         x_desired[0:3] = p_desired
#         x_desired[6:9] = v_desired
#         self.x_desired = x_desired

#         u = -self.K @ (self.x_hat - self.x_desired)

#         y = pos_markers

#         self.x_hat += self.dt * (self.A @ self.x_hat + self.B @ u - self.L @ (self.C @ self.x_hat - y))
        
#         tau_x = u[0]
#         tau_y = u[1]
#         tau_z = u[2]
#         f_z = u[3] + self.f_z_e

#         return tau_x, tau_y, tau_z, f_z

In [ ]:
# #PERFORMANCE GOAL 3 - No Yaw
# # up 1m, 5m at 1m/s, stop, yaw 90 deg left, 5m forward, land vertically < 0.01m/s speed
# #Uses different LQR values -- See comment in LQR Section

# class Controller:
#     def __init__(self):

#         self.dt = 0.01

#         self.variables_to_log = ['x_hat', 'x_desired']

#         self.A = A
#         self.B = B
#         self.C = C
#         self.K = K
#         self.L = L


#         self.tau_x_e = 0
#         self.tau_y_e = 0
#         self.tau_z_e = 0
#         self.f_z_e = 9.81*0.5
       
#         self.phase=0


    
#     def get_color(self):

#         return [
#             1. , # <-- how much red (between 0 and 1)
#             0., # <-- how much green (between 0 and 1)
#             0.25, # <-- how much blue (between 0 and 1)
#         ]

#     def reset(self, p_x=0, p_y=0, p_z=0, yaw=0):
#         self.x_hat = np.array([p_x, p_y, p_z, yaw, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
#         self.x_desired = np.zeros(12, dtype=float)


#         self.yaw_initial=yaw
        

#     def run(self, **kwargs):



#         mocap1=kwargs['mocap_1']
#         mocap2=kwargs['mocap_2']
#         mocap3=kwargs['mocap_3']
#         mocap4=kwargs['mocap_4']
        
#         dt = kwargs.get('dt', 0.01)


#         pos_markers = np.array([mocap1[0],mocap1[1],mocap1[2],
#                                 mocap2[0],mocap2[1],mocap2[2],
#                                 mocap3[0],mocap3[1],mocap3[2],
#                                 mocap4[0],mocap4[1],mocap4[2],])
        
#         x_desired = np.zeros(12)
#         pos = self.x_hat[0:3]


#         p_desired = self.x_hat[0:3]
#         v_desired = np.zeros(3)

# # -----------------------------------------        
#         #Getting the drone to 1m 
        
#         e_max = 0.01

#         if self.phase==0:
#             final_pos=np.array([0, 0, 1])
#             if np.linalg.norm(final_pos - pos) > 0.1:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 0.05:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.05 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.01:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.007 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))  

#             else:
#                 p_desired = final_pos
#                 v_desired = np.zeros(3) 
#                 self.phase = 1
                
# # -----------------------------------------
#         #5m forward at 1 m/s
        
#         elif self.phase==1:
#             final_pos=np.array([0,5,1])
#             if np.linalg.norm(final_pos - pos) > 1.5:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 1 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 1:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos)) 
#             elif np.linalg.norm(final_pos-pos) > 0.5:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.1 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.3:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))   
#             else:
#                 p_desired = final_pos
#                 v_desired = 0.0001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 self.phase = 2

# # -----------------------------------------
#         #5m left
        
#         if self.phase==2:
#             final_pos=np.array([5, 5, 1])
#             if np.linalg.norm(final_pos - pos) > 1.5:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 1 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 1:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos)) 
#             elif np.linalg.norm(final_pos-pos) > 0.5:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.1 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.3:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             else:
#                 p_desired = final_pos
#                 v_desired = 0.0001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 self.phase = 3

# # -----------------------------------------
#         #Land vertically <0.01 m/s
        
#         if self.phase==3:
#             final_pos=np.array([5, 5, 0])
#             if np.linalg.norm(final_pos - pos) > 0.5:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.2 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 0.2:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.05 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.001:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.001 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))  

#             else:
#                 p_desired = final_pos
#                 v_desired = np.zeros(3)
                
# # ----------------------------------------- 
        
#         x_desired[0:3] = p_desired
#         x_desired[6:9] = v_desired
#         self.x_desired = x_desired


        
#         u = -self.K @ (self.x_hat - self.x_desired)


        
#         y = pos_markers

        
#         self.x_hat += self.dt * (self.A @ self.x_hat + self.B @ u - self.L @ (self.C @ self.x_hat - y))
        
#         tau_x = u[0]
#         tau_y = u[1]
#         tau_z = u[2]
#         f_z = u[3] + self.f_z_e


#         return tau_x, tau_y, tau_z, f_z


In [ ]:
# # #PERFORMANCE GOAL 3 - Yaw, no linear
# # up 1m, 5m at 1m/s, stop, yaw 90 deg left, 5m forward, land vertically < 0.01m/s speed

# class Controller:
#     def __init__(self):

#         self.dt = 0.01

#         self.variables_to_log = ['x_hat', 'x_desired']

#         self.A = A
#         self.B = B
#         self.C = C
#         self.K = K
#         self.L = L


#         self.tau_x_e = 0
#         self.tau_y_e = 0
#         self.tau_z_e = 0
#         self.f_z_e = 9.81*0.5
       
#         self.phase=0
    
#     def get_color(self):

#         return [
#             1. , # <-- how much red (between 0 and 1)
#             0., # <-- how much green (between 0 and 1)
#             0.25, # <-- how much blue (between 0 and 1)
#         ]

#     def reset(self, p_x=0, p_y=0, p_z=0, yaw=0):
#         self.x_hat = np.array([p_x, p_y, p_z, yaw, 0, 0, 0, 0, 0, 0, 0, 0], dtype=float)
#         self.x_desired = np.zeros(12, dtype=float)


#         self.yaw_initial=yaw
        

#     def run(self, **kwargs):

#         mocap1=kwargs['mocap_1']
#         mocap2=kwargs['mocap_2']
#         mocap3=kwargs['mocap_3']
#         mocap4=kwargs['mocap_4']
        
#         dt = kwargs.get('dt', 0.01)


#         pos_markers = np.array([mocap1[0],mocap1[1],mocap1[2],
#                                 mocap2[0],mocap2[1],mocap2[2],
#                                 mocap3[0],mocap3[1],mocap3[2],
#                                 mocap4[0],mocap4[1],mocap4[2],])
        
#         x_desired = np.zeros(12)
#         pos = self.x_hat[0:3]

#         ori=self.x_hat[3:6]
#         ori_desired=self.x_hat[3:6] #Yaw and hover works with this, position change doesn't
#         w_desired = np.zeros(3)
#         p_desired = self.x_hat[0:3]
#         v_desired = np.zeros(3)

# # ----------------------------------------- 
#         #Getting the drone to 1m 
        
#         e_max = 0.01

#         if self.phase==0:
#             final_pos=np.array([0, 0, 1])
#             if np.linalg.norm(final_pos - pos) > 0.1:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.5 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#             elif np.linalg.norm(final_pos-pos) > 0.05:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.05 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))    
#             elif np.linalg.norm(final_pos-pos) > 0.01:
#                 p_desired = pos +  e_max * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 v_desired = 0.007 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))  

#             else:
#                 p_desired = final_pos
#                 v_desired = np.zeros(3) 
#                 # v_desired = 0.007 * ((final_pos - pos) / np.linalg.norm(final_pos - pos))
#                 self.phase = 1

# # ----------------------------------------- 
#         #Yaw left

#         elif self.phase==1: #yaw 90 deg left NOT WORKING YET
#             yaw_desired=pi/2 #90 degrees
        
#             yaw_change=self.x_hat[3]-self.yaw_initial
        
#             final_ori=np.array([yaw_desired,0,0])
        
#             e_ori=0.01

#             if abs(yaw_change) < (pi/2):
#                 ori_desired = ori +  e_ori * ((final_ori - ori) / np.linalg.norm(final_ori - ori))
#             else:              
#                 w_desired = np.zeros(3)
#                 self.phase=4 

# # ----------------------------------------- 
        
#         x_desired[0:3] = p_desired
#         x_desired[6:9] = v_desired
#         x_desired[3:6] = ori_desired
#         x_desired[9:12] = w_desired
#         self.x_desired = x_desired

#         u = -self.K @ (self.x_hat - self.x_desired)

#         y = pos_markers
        
#         self.x_hat += self.dt * (self.A @ self.x_hat + self.B @ u - self.L @ (self.C @ self.x_hat - y))
        
#         tau_x = u[0]
#         tau_y = u[1]
#         tau_z = u[2]
#         f_z = u[3] + self.f_z_e

#         return tau_x, tau_y, tau_z, f_z

In [ ]:
controller1=Controller()
team_list = [""]
sim = ae352_quadrotor_new.Quadrotor_Sim(n_quadrotors=1,
                                    team_list=team_list,
                                    planet="Earth",
                                    use_keyboard=True,
                                    # visualization=True, #UNCOMMENT THIS AND COMMENT OUT BELOW TO RUN VISUALS
                                    visualization=False,
                                    visualization_fr=45.,
                                    animation=False,
                                    verbose=True)

In [ ]:
controllers = [controller1]
(track_stats, data) = sim.run(controllers,
                              sensor_noise=0.,
                              collect_data=True,
                              max_time=40.0, #change time based on performance goal
                              verbose=True) 

In [ ]:
# Create a figure that has two plots in it
import matplotlib.ticker as ticker
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=False)

# Convert the position data to array
colors = ['k','r','g','b','m','c','tab:orange','tab:purple','tab:brown','tab:pink','tab:gray']
for i in range(len(data)):
    
    # Get the position
    pos = np.array(data[i]['position'])
    
    # Create a plot of the ground path of the quadrotor
    axes[0].plot(pos[:,0], pos[:,1], c=colors[i], lw=2.5, label=str(i+1))
    
    # Create a plot of the altitude versus time
    axes[1].plot(data[0]['time'], pos[:,2], c=colors[i], lw=2.5, label=str(i+1))

# Make plot 0 pretty
axes[0].set_xlabel("X Position [meters]", fontsize=12)
axes[0].set_ylabel("Y Position [meters]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].set_title("Ground Track", fontsize=14)
axes[0].set_aspect('equal', 'box')
axes[0].legend(bbox_to_anchor=(1.16, 1.05), fancybox=True, shadow=True)
axes[0].grid()

# Make plot 1 pretty
axes[1].set_xlabel("Time [seconds]", fontsize=12)
axes[1].set_ylabel("Altitude [meters]", fontsize=12)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].set_title("Altitude vs. Time", fontsize=14)
axes[1].legend(bbox_to_anchor=(1., 1.05), fancybox=True, shadow=True)
axes[1].grid()

# Show the figure
fig.tight_layout(pad=2.0)
plt.show()

In [ ]:
# TRAJECTORY TRACKING

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

for i in range(len(data)):
    
    # Get the position
    pos = np.array(data[i]['position'])

    drone_x = pos[:,0]
    drone_y = pos[:,1]
    drone_z = pos[:,2]



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


ax.plot3D(drone_x, drone_y, drone_z, label='Drone Trajectory')


ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.legend()

plt.show()

In [ ]:
import matplotlib.ticker as ticker

fig, (ax_pos, ax_vel, ax_ori, ax_ang) = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# -----------------------------------------

# Position

for i in range(len(data)):
    
    t = np.array(data[i]['time'])
    
    pos = np.array(data[i]['position'])

    p_x = pos[:,0]
    p_y = pos[:,1]
    p_z = pos[:,2]

ax_pos.plot(t, p_x, label='Position_x', linewidth=3, color='C0')
ax_pos.plot(t, p_y, label='Position_y', linewidth=3, color='C1')
ax_pos.plot(t, p_z, label='Position_z', linewidth=3, color='C2')

ax_pos.grid()
ax_pos.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_pos.tick_params(labelsize=14)

# -----------------------------------------

# Velocity

for i in range(len(data)):
    
    t = np.array(data[i]['time'])
    
    vel = np.array(data[i]['velocity'])

    v_x = vel[:,0]
    v_y = vel[:,1]
    v_z = vel[:,2]

    v_mag = np.sqrt(v_x**2 + v_y**2 + v_z**2)
    v_avg = np.cumsum(v_mag) / np.arange(1, len(v_mag)+1)

ax_vel.plot(t, v_x, label='V_x (s)', linewidth=3, color='C0')
ax_vel.plot(t, v_y, label='V_y (s)', linewidth=3, color='C1')
ax_vel.plot(t, v_z, label='V_z (s)', linewidth=3, color='C2')
ax_vel.plot(t, v_avg, label='V_Average (s)', linewidth=3, color='C3')
ax_vel.grid()
ax_vel.yaxis.set_major_locator(ticker.MultipleLocator(base=0.1))
ax_vel.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_vel.tick_params(labelsize=14)

# -----------------------------------------

# Orientation

for i in range(len(data)):
    
    t = np.array(data[i]['time'])
    
    ori = np.array(data[i]['orientation'])

    yaw = ori[:,0]
    pitch = ori[:,1]
    roll = ori[:,2]

ax_ori.plot(t, yaw, label='Yaw', linewidth=3, color='C0')
ax_ori.plot(t, pitch, label='Pitch', linewidth=3, color='C1')
ax_ori.plot(t, roll, label='Roll', linewidth=3, color='C2')
ax_ori.grid()
ax_ori.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_ori.tick_params(labelsize=14)
fig.tight_layout()

# -----------------------------------------

#Angular Velocity

for i in range(len(data)):
    
    t = np.array(data[i]['time'])
    
    ang = np.array(data[i]['angular velocity'])

    ang_x = ang[:,0]
    ang_y = ang[:,1]
    ang_z = ang[:,2]

ax_ang.plot(t, ang_x, label='w_x', linewidth=3, color='C0')
ax_ang.plot(t, ang_y, label='w_y', linewidth=3, color='C1')
ax_ang.plot(t, ang_z, label='w_z', linewidth=3, color='C2')
ax_ang.grid()
ax_ang.legend(fontsize=16, bbox_to_anchor=(1., 1.))
ax_ang.tick_params(labelsize=14)
fig.tight_layout()
plt.show()